In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser, BaseOutputParser
from textwrap import dedent

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
import httpx
import json
import requests
import pandas as pd
import numpy as np
import re

from langchain_pymupdf4llm import PyMuPDF4LLMLoader, PyMuPDF4LLMParser
import pymupdf4llm
import requests
import uuid
from openai import OpenAI

import time
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True))

BASE_URL = "https://gigachat.devices.sberbank.ru/api/v1"
#base_gg = "https://router.huggingface.co/v1"
#"http://a6k2.dgx:34000/v1"

BASE_URL = "https://router.huggingface.co/v1"

#"qwen3-32b"
# gigachat/GigaChat
MODEL_NAME = os.getenv("MODEL_NAME", 'GigaChat')



MODEL_NAME_HF ='meta-llama/Llama-3.3-70B-Instruct'
MODEL_NAME_HF ='Qwen/Qwen3-32B'

MODEL_TEMPERATURE_HF =1

os.environ["BASE_URL"] = "https://gigachat.devices.sberbank.ru/api/v1"
os.environ["MODEL_NAME"] ='GigaChat'
#os.environ["OPENAI_API_KEY"] = os.getenv("GIGACHAT_AUTH")

os.environ['TEMPERATURE'] = '0'

url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
# Создадим идентификатор UUID (36 знаков)
rq_uid = str(uuid.uuid4())
payload={
  'scope': 'GIGACHAT_API_PERS'
}
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json',
  'RqUID': rq_uid,
  'Authorization': f'Basic {os.getenv("GIGACHAT_AUTH")}'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)
os.environ["OPENAI_API_KEY"] = response.json()['access_token']


from trialmind.pubmed import pmid2papers, PubmedAPIWrapper, pmid2biocxml, parse_bioc_xml
from trialmind.api import StudyCharacteristicsExtraction, ScreeningCriteriaGeneration,LiteratureScreening
from trialmind.retrievers import split_text_into_chunks
import extract

%load_ext autoreload
%autoreload 2


C:\Users\user\anaconda3\envs\llms\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


Consider using the pymupdf_layout package for a greatly improved page layout analysis.


C:\Users\user\anaconda3\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### clinical trials

In [2]:
file_path = "docs/CC_215_L00_DL_fusion_fixed_oncobox_ru.pdf"
fin_condition, treatements_eng = extract.info_from_doc(file_path)
fin_condition, treatements_eng

('Colorectal cancer',
 ['Crizotinib',
  'Loratinib',
  'Necitumumab',
  'Nimotuzumab',
  'Panitumumab',
  'Cetuximab',
  'Brigatinib',
  'Ramucirumab',
  'Pomalidomide',
  'Tamoxifen',
  'Denosumab',
  'Duvelisib',
  'Inavilisib',
  'Bevacizumab',
  'Anastrozole',
  'Exemestane'])

1. загрузить док
2. взять 1-ую страницу; рег.выражениями взять диагноз
-- с ЛЛМ перевести на англ.
3. найти страницу с таблицей; или взять таблицы с PyMuPDFLoader
4. извлечь таблицу

In [8]:
' OR '.join(treatements_eng)

'Crizotinib OR Loratinib OR Necitumumab OR Nimotuzumab OR Panitumumab OR Cetuximab OR Brigatinib OR Ramucirumab OR Pomalidomide OR Tamoxifen OR Denosumab OR Duvelisib OR Inavilisib OR Bevacizumab OR Anastrozole OR Exemestane'

In [5]:
treatements_eng[5], f'''"{fin_condition}"'''

('Cetuximab', '"Colorectal cancer"')

In [7]:
all_studies = extract.get_clinicaltrials(f'''"{fin_condition}"''', 
                                      #' OR '.join(treatements_eng), 
                                       treatements_eng[5],  
                                      max_studies=4)
all_studies

ZVNj7o2Elu8o3lppXcuzsbrumpOQJJxsYfWj3_g
ZVNj7o2Elu8o3lppXcuzsbrumpOQJJxobPKj1fg
ZVNj7o2Elu8o3lppXcuzsbrumpOQJJxmYvGg3fg
ZVNj7o2Elu8o3lppXcuzsbrumpOQJJxuZ_Cg3_AW


,results,id,title,status,conditions,study_type,phases,interventions,outcomes
0,False,NCT06959589,"The Efficacy and Safety of IBI351, Cetuximab β...",NOT_YET_RECRUITING,"[Colorectal Cancer Metastatic, KRAS G12C Mutat...",INTERVENTIONAL,[PHASE2],"[ibi351, cetuximab β combined with folfiri]",NaN
1,False,NCT02484833,Erbitux MEtastatic Colorectal Cancer Strategy ...,COMPLETED,[Antineoplastic Agents],INTERVENTIONAL,[PHASE3],"[cetuximab, folfiri]",NaN
2,False,NCT06358430,Dose Escalation and Expansion Study of TROP2 C...,RECRUITING,"[Colorectal Cancer, Minimal Residual Disease]",INTERVENTIONAL,[PHASE1],"[fludarabine phosphate, cyclophosphamide, cetu...",NaN
3,True,NCT00703976,"Radiation, Cetuximab and Pemetrexed With or Wi...",COMPLETED,[Cancer],INTERVENTIONAL,[PHASE2],"[bevacizumab, cetuximab, pemetrexed]","[{'type': 'PRIMARY', 'title': '2-year Progress..."
4,True,NCT00835679,Cetuximab and/or Dasatinib in Patients With Co...,TERMINATED,"[Liver Metastases, Mucinous Adenocarcinoma of ...",INTERVENTIONAL,[EARLY_PHASE1],[dasatinib],"[{'type': 'PRIMARY', 'title': 'Patients With a..."
5,False,NCT00719199,Study of FOLFIRI Plus Cetuximab Plus IMO-2055 ...,TERMINATED,[Colorectal Cancer],INTERVENTIONAL,[PHASE1],"[imo-2055, cetuximab, folfiri]",NaN
6,False,NCT04513951,AVELUMAB and CETUXIMAB and mFOLFOXIRI as Initi...,COMPLETED,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[5fluorouracil, l-leucovorin, irinotecan, oxal...",NaN
7,False,NCT06856187,Second-line Standard Treatment Sequential TAS-...,RECRUITING,[Metastatic Colorectal Cancer (CRC)],INTERVENTIONAL,[PHASE2],"[tas-102+bevacizumab+local treatment, standard...",NaN
8,False,NCT01167725,Standard Therapy With or Without Surgery and M...,TERMINATED,[Colorectal Cancer],INTERVENTIONAL,[PHASE3],"[folfiri regimen, folfox regimen, capecitabine...",NaN
9,False,NCT02029326,Biomarker Analysis in Metastatic Colorectal Ca...,COMPLETED,[Colorectal Cancer],OBSERVATIONAL,NaN,[],NaN


In [8]:
chosen = all_studies[all_studies.results==True]
chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x \
                                                           if(obj.get('unitOfMeasure','').lower() in ['percentage of participants','participants'])])
chosen.shape

C:\Users\user\AppData\Local\Temp\ipykernel_3488\2348048961.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x \


(3, 10)

In [24]:
[i.split('.')[-1] for i in all_studies.columns]

['hasResults',
 'nctId',
 'briefTitle',
 'overallStatus',
 'briefSummary',
 'conditions',
 'studyType',
 'interventions']

In [29]:
base = 'https://clinicaltrials.gov/api/v2/studies'  # new API

# fields for CSV and JSON: https://clinicaltrials.gov/data-api/about-api/csv-download

# new API fields - for JSON
extract_fields = [
    'NCTId',
    'Condition',
    'BriefTitle',
    'HasResults',
    "Intervention",
    #'PrimaryOutcome',
    'OutcomeMeasuresModule',
    #'NumPhases',
    'Phase',
    'OverallStatus',
    'StudyType',
    'BriefSummary',
    #'LastKnownStatus',

]
#OutcomeMeasuresModule 
# new API fields
params = {
    'fields': ",".join(extract_fields), 
    'query.term': '''"Colorectal Cancer"''',
    'query.intr': 'Cetuximab',
    #'filter.overallStatus':'',
    'pageSize': 1, # maximum number of !!studies!! to return in response
    #'aggFilters':'results:with',
    'format': 'json', 
    #'pageToken': None  # first page doesn't need it
}    
all_studies = []
results_studies = []
no_results_studies = []
next_page_token='a'
page_temp = 0
while next_page_token and page_temp<2:

    #print(f'--- page: {page} ---')

    response = requests.get(base, params=params)

    if not response.ok:
        print('response.text:', response.text)
        break

    data = response.json()
    #print(data.keys())
    data_responses = data['studies']
    #print(data_responses)

    all_studies = all_studies+data_responses
    '''
    for index, item_main in enumerate(data_responses, 1):
        #print(item)
        item = item_main['protocolSection']['identificationModule']
        #print(item)

        print(f'{page:2},{index:4}:', item_main['protocolSection']['statusModule']['overallStatus'], 
              #item_main['protocolSection']['statusModule']['lastKnownStatus'],
              item_main['protocolSection']['designModule']['phases'],
              #item_main['protocolSection']['designModule']['numPhases'],
              #item_main['resultsSection']['outcomeMeasuresModule']['outcomeMeasures'], 
              item['nctId'], item['briefTitle'])
        '''

    try:
    # next page
    #next_page_token = response.headers.get('x-next-page')  # (probably) for `CSV`
        next_page_token = data['nextPageToken']                 # for `JSON`
        print(data['nextPageToken'])
    #print('x-next-page', next_page_token)    
        params['pageToken'] = next_page_token
    except KeyError:
        next_page_token=''
        pass
    page_temp+=1

all_studies = pd.json_normalize(all_studies)
print(all_studies.columns)
'''
['hasResults','nctId','briefTitle','overallStatus',
 'briefSummary','conditions','studyType','interventions']
'''

all_studies.columns = [i.split('.')[-1] for i in all_studies.columns]

if 'outcomeMeasures' not in all_studies.columns:
    all_studies['outcomeMeasures']=None
    
all_studies    

ZVNj7o2Elu8o3lppWovzpbrumpOQJJxmZ_Wp
ZVNj7o2Elu8o3lppWovzpbrumpOQJJxsZ_at0A
Index(['hasResults', 'protocolSection.identificationModule.nctId',
       'protocolSection.identificationModule.briefTitle',
       'protocolSection.statusModule.overallStatus',
       'protocolSection.descriptionModule.briefSummary',
       'protocolSection.conditionsModule.conditions',
       'protocolSection.designModule.studyType',
       'protocolSection.designModule.phases',
       'protocolSection.armsInterventionsModule.interventions',
       'resultsSection.outcomeMeasuresModule.outcomeMeasures'],
      dtype='object')


,hasResults,nctId,briefTitle,overallStatus,briefSummary,conditions,studyType,phases,interventions,outcomeMeasures
0,True,NCT03563157,QUILT 3.071: NANT Colorectal Cancer (CRC) Vaccine,TERMINATED,QUILT 3.071 NANT Colorectal Cancer (CRC) Vacci...,"[Colorectal Cancer Metastatic, mCRC]",INTERVENTIONAL,"[PHASE1, PHASE2]","[{'type': 'BIOLOGICAL', 'name': 'Aldoxorubicin...","[{'type': 'PRIMARY', 'title': 'Number of Parti..."
1,True,NCT00321100,"Combination of Cetuximab, Capecitabine, and Ox...",TERMINATED,The purpose of this study is to determine the ...,[Colorectal Neoplasms],INTERVENTIONAL,[PHASE2],"[{'type': 'DRUG', 'name': 'bevacizumab', 'desc...","[{'type': 'PRIMARY', 'title': 'Objective Respo..."


In [9]:
chosen[chosen.res_with_part.str.len()>0].shape

(2, 10)

In [10]:
chosen.res_with_part.str.len()

3      2
4      0
11    10
Name: res_with_part, dtype: int64

In [11]:
chosen[chosen.res_with_part.str.len()>0].head(2)

,results,id,title,status,conditions,study_type,phases,interventions,outcomes,res_with_part
3,True,NCT00703976,"Radiation, Cetuximab and Pemetrexed With or Wi...",COMPLETED,[Cancer],INTERVENTIONAL,[PHASE2],"[bevacizumab, cetuximab, pemetrexed]","[{'type': 'PRIMARY', 'title': '2-year Progress...","[{'type': 'PRIMARY', 'title': '2-year Progress..."
11,True,NCT01787006,Definitive Radiochemotherapy Plus/Minus Cetuxi...,COMPLETED,[Esophageal Cancer],INTERVENTIONAL,[PHASE2],"[cetuximab, cisplatin, 5-fu]","[{'type': 'PRIMARY', 'title': 'Rate of Partici...","[{'type': 'PRIMARY', 'title': 'Rate of Partici..."


In [12]:
from trialmind.llm_utils.openai_async import batch_call_openai

### outcome results
Взяли все результаты, отдельно сохранили испытания с результатами, и з них отбираем рез-ты только с unit==participants

filter by interventional study type?

In [13]:
at_once = False

chosen = all_studies[all_studies.results==True]
chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x \
                                                          if(obj.get('unitOfMeasure','').lower() in ['percentage of participants','participants'])])

to_work = chosen[chosen.res_with_part.str.len()>0]
if to_work.shape[0]:
    messages = []
    for i in to_work.res_with_part.values[:2]:
        n_outcomes = len(i)
        print(n_outcomes)
        messages.append([{'role':'system', 'content':f'You are a helpful medical assistant. Answer with a concise description of the efficiency statistics for EACH of {n_outcomes} provided outcome measures /no_think'},
                {'role':'user', 'content':f"{i}"}])
    if at_once:
        results = batch_call_openai(messages, os.getenv('MODEL_NAME'), int(os.getenv('TEMPERATURE')), thinking=False)
    else:
        results = []
        for i in messages:
            answer = extract.use_llm(os.getenv('MODEL_NAME'),i)
            results.append(answer)
    for i in results:
        print(i)
        print('-----')

C:\Users\user\AppData\Local\Temp\ipykernel_3488\967050974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x \


2
10
**Outcome Measure 1: 2-Year Progression-Free Survival (PFS)**

- **Efficiency Statistics:** 
  - Group OG000: 79% (90% CI: 69%-92%)
  - Group OG001: 75% (90% CI: 64%-88%)
  
**Outcome Measure 2: 2-Year Overall Survival (OS)**

- **Efficiency Statistics:** 
  - All Participants (Overall Study): 88% (95% CI: 81%-96%)
  - Group OG001: 91% (95% CI: 82%-100%)
  - Group OG002: 87% (95% CI: 77%-98%)
-----
Here are the efficiency statistics for each outcome measure:

1. **Rate of Participants Alive at 2 Years**  
   - Group OG000: 71%, CI: 55–87%
   - Group OG001: 53%, CI: 36–71%

2. **Rate of Participants Alive at 1 Year**  
   - Group OG000: 74%, CI: 59–90%
   - Group OG001: 70%, CI: 54–86%

3. **Rate of Participants Alive Without Progression at 1 Year**  
   - Group OG000: 64%, CI: 47–82%
   - Group OG001: 58%, CI: 40–75%

4. **Rate of Participants Alive Without Progression at 2 Years**  
   - Group OG000: 56%, CI: 37–75%
   - Group OG001: 44%, CI: 26–62%

5. **Number of Participants W

In [59]:
temperature=0
def results_ct(all_studies):
    chosen = all_studies[all_studies.results==True]
    chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x if(obj.get('unitOfMeasure','') == 'Participants')])
    
    to_work = chosen[chosen.res_with_part.str.len()>0].iloc[:2]
    if to_work:
        outcome_participants = [obj for item_main in chosen.outcomes.values for obj in item_main 
                            if(obj.get('unitOfMeasure','') == 'Participants')] 
        messages = []
        for i in to_work.res_with_part.values:
            messages.append([{'role':'system', 'content':'You are a helpful medical assistant. Extract information about the efficiency statistics from each outcome measure. Give: 1) a concise description of results with percentages and 2) a short description of study objectives /no_think'},
                    {'role':'user', 'content':f"{i}"}])

        results = batch_call_openai(messages, os.getenv('MODEL_NAME'), int(os.getenv('TEMPERATURE')), thinking=False)
        return results
    else:
        return None

res = results_ct(all_studies)
for i in res:
    print(i)
    print('-----')

C:\Users\user\AppData\Local\Temp\ipykernel_17840\480369459.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen['res_with_part'] = chosen.outcomes.apply(lambda x: [obj for obj in x if(obj.get('unitOfMeasure','') == 'Participants')])


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [19]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful medical assistant. Extract information about the efficiency statistics from each outcome measure. Give a concise description with percentages /no_think"),
    ("human", "{text}"),
    # ("ai", "Good.")
])

chain2 = prompt2 | llm_reasoning
response_res = await chain2.ainvoke({"text": outcome_participants[0]})
print(response_res.content)

NameError: name 'llm_reasoning' is not defined

### results in a given format?

In [ ]:

class Info(BaseModel):
    diagnose: str = Field(..., description="База клинических испытаний")
    treatments: str = Field(..., description="Высшие позиции рейтинга таргетных препаратов")

parser = PydanticOutputParser(pydantic_object=Info)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        dedent(
            """
            You are a helpful assistant. 
            Your goal is to extract related information from the text provided by the user.
            Return the information in the following JSON-format.
            {fmt}
            """
        )
    ),
    (
        "human", 
        dedent(
            """
            Text: {text}"
            """
        )
    ),
])
chain = (
    prompt 
    | llm_reasoning 
    | parser
)


response = await chain.ainvoke({"text": text, "fmt": parser.get_format_instructions()})
print(response)

### papers

In [3]:
fin_condition = 'Colorectal cancer'
treatements_eng =  ['Crizotinib','Lorlatinib',]
pubmed_api_key= '6892c4129cef143ff92d11533848d2e0d908'#os.getenv("PUBMED_API_KEY", '')

In [4]:
search_api = PubmedAPIWrapper()
# page_size is the max number of records to return!!!! not pages!
tmp_inputs = {
        "page_size": 3,
        "keyword_map": {'conditions':[fin_condition], 
                        'treatments':[treatements_eng[0]]
                       },
        "keywords": {
            "OPERATOR": 'AND'
        }
}

time.sleep(.5)
response = search_api.build_search_query_and_get_pmid(tmp_inputs, 
                                                      api_key=pubmed_api_key)
response

(['40526090', '41213063', '41268440'],
 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=(Colorectal cancer)+AND+(Crizotinib)&retmax=3&retmode=json&api_key=6892c4129cef143ff92d11533848d2e0d908',
 '67')

In [4]:
df_papers = pmid2papers(pmid_list=['40526090', '41213063', '41268440'],#response[0], 
                        api_key=pubmed_api_key)
df_papers[0].head()

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract
0,40526090,Oncotarget,2025,Jun,17,Case Report WIN-MTB-2023001 WIN International ...,"Journal Article, Case Reports","Alberto Hernando-Calvo, Razelle Kurzrock, Nadi...",Heavily pretreated metastatic colorectal cance...
1,41213063,"Journal of environmental pathology, toxicology...",2025,,,Radiotherapy-Related Autophagy Genes Predict P...,Journal Article,"Mao Mao, Yichao Zhuang, Yan Chen",Numerous genes have been associated with color...
2,41268440,Frontiers in pharmacology,2025,,,Case Report: HGF and NF1 mutations as putative...,"Case Reports, Journal Article","Zhitao Chen, Shan Luo, Yangjun Gu, Qiyong Li",Hepatocellular carcinoma (HCC) is a highly agg...


In [5]:
fin_condition, treatements_eng

('Colorectal cancer', ['Crizotinib', 'Lorlatinib'])

### extract results

In [6]:
fin_condition = 'Colorectal cancer'
treatements_eng =  ['Crizotinib','Lorlatinib',]

In [7]:
os.getenv("GIGACHAT_AUTH")

'MDE5YmM2MjctOWU5ZS03OWE0LWE0OGYtY2FhMTU4MzYxZDgzOjg2MjAwMDgzLThkNzMtNGU0ZS1hNzIxLTM3YWY5ZWY4ZmZlNg=='

### get sber api

C:\Users\user\anaconda3\envs\llms\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


'eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.AUismEA_iypp1Wa3hGSGS_CGMe9ZNJYFoaPG43Jz94F53ubk4aa_UdZU5JJGTB7M2A-gbyGdaeQVLK6KEEjVdiAITdOa4Cazz9khlamOsPv9kZKS0XQlWAoNJ3WejWx9iL5za4Y4FKfROi7MaNeCZQu8pO-DslBYqBkoJmpg_tSuelDE9ldFzwFzAM_Cc2PsyJTBfkWxhi3qdqhJ6kE77mp0mvVULOxvNMOSlI9b0mmIm3q4kJemumusdKKo8FTZCn2Ub6VbvAnebVUuYywE4OctKMbOe0MjoYDh1VO8_jcW6dgANcsPNfwHKRnD_UqVs85v6_bnOhSIimi45nqH5w.ziBWyLF-sztnPom5QV8fPg.a7t8GaPBh9wuxMLMO-qkwuTzlqlQEgeDQXLEzPoOggxuVb-5ezFfAMLvV1UAgAyFhJEGF-RuEycOy2zNRAvTGVyh9i6xVwYu88y-ZkS-1wo7vPAsyBhZJuGH6rqJCbnAeqT9gF0ZTjAg2w35WAEpPkKpMaFhILyIh85Tnc8q1i61Fo-qwVke66uMq5vwNiIREH5hBi1YmliWgPg5BhIVfyGaYNwa7Lsd8ApxBLNTxV1aW3XqxoqOcOjkPUmltypJ0_kr27kqCjmhw-PVC-BV6UVB5liMbdbg-2W9yR6pZc1lZ6_Wl9b9YnpJBm2jxY1oe8Kc0-buTLNC6-V52oftlk7-hKm-nRUI2L-3A3iDIj6dqsNRE2q_shOvvG4eh9ArULHISG_w4wjVed2O_8nivDWjinE47LfgstH0_Jl-c_WXbsV6h82f3fOBauvgHZeBo4PYikMFZkSdMfhezjYF2cRdY__r5eUB_rvFVK7wZ5BwcZdng4JibdXJvCA-Sy4LWbParXM6puf_BlettkEUrYIWsd2B5uED1-4vQ2g9l8MU8

In [41]:
# test the apis

api = ScreeningCriteriaGeneration()
ecs = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    num_title_criteria=2,
    num_abstract_criteria=2,
    thinking=False,
)

call_openai https://gigachat.devices.sberbank.ru/api/v1
api_call_single https://gigachat.devices.sberbank.ru/api/v1/
```json
{
  "ELIGIBILITY_ANALYSIS": [
    "Patients must have colorectal cancer — Studies focusing on other types of cancers would not contribute relevant data.",
    "Studies involving patients receiving crizotinib as an intervention — Exclusion of studies using different treatments ensures consistency in the analysis.",
    "Comparative studies where crizotinib is compared against another treatment or placebo — Inclusion of comparative studies allows for direct comparison of outcomes between crizotinib and alternative therapies.",
    "Studies reporting outcomes related to efficacy, safety, or survival — The focus is on evaluating effectiveness and adverse effects of crizotinib.",
    "Published studies in peer-reviewed journals — Ensures quality and reliability of the evidence."
  ],
  "TITLE_CRITERIA": [
    "Does the title mention 'colorectal cancer'?",
    "Does th

In [42]:
ecs

{'title_criteria': ["Does the title mention 'colorectal cancer'?",
  "Does the title include 'crizotinib'?"],
 'content_criteria': ['Is there a clear description of patient population with colorectal cancer?',
  'Are the results reported in terms of efficacy, toxicity, or survival rates associated with crizotinib use?'],
 'eligibility_analysis': ['Patients must have colorectal cancer — Studies focusing on other types of cancers would not contribute relevant data.',
  'Studies involving patients receiving crizotinib as an intervention — Exclusion of studies using different treatments ensures consistency in the analysis.',
  'Comparative studies where crizotinib is compared against another treatment or placebo — Inclusion of comparative studies allows for direct comparison of outcomes between crizotinib and alternative therapies.',
  'Studies reporting outcomes related to efficacy, safety, or survival — The focus is on evaluating effectiveness and adverse effects of crizotinib.',
  'Publ

In [11]:

papers = df_papers[0].iloc[:3]["Title"] + ": " + df_papers[0].iloc[:3]["Abstract"].fillna("") # important to fillna
papers = papers.tolist()

In [16]:
'''
title_criteria = ["Does the title mention 'colorectal cancer'?",
"Does the title mention 'crizotinib'?",
"Is there any indication in the title that the study compares crizotinib to another treatment?"
]

content_criteria = ["Are the methods section and results clearly described in the paper?",
"Does the study specify the type of colorectal cancer (e.g., adenocarcinoma)?",
"Were the outcomes measured over a sufficient duration to assess long-term effects?"
  ]
'''
title_criteria = ecs['title_criteria']
content_criteria = ecs['content_criteria']
title_criteria+content_criteria

NameError: name 'ecs' is not defined

In [12]:
title_criteria = ["Does the title mention 'colorectal cancer'?",
"Does the title mention 'crizotinib'?",
"Is there any indication in the title that the study compares crizotinib to another treatment?"
]

content_criteria = ["Are the methods section and results clearly described in the paper?",
"Does the study specify the type of colorectal cancer (e.g., adenocarcinoma)?",
"Were the outcomes measured over a sufficient duration to assess long-term effects?"
  ]

### lit screen

In [15]:
from trialmind.llm import _batch_inputs_to_messages

In [22]:
from trialmind.prompts.screening import LITERATURE_SCREENING_FC
population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
intervention = f"{treatements_eng[0]}",
comparator = "",
outcome = "",
criteria = title_criteria + content_criteria
criteria_text = [f"{idx+1}. {c}" for idx, c in enumerate(criteria)]
n_criteria = len(criteria_text)
    
batch_inputs = []
for paper in papers[:1]:
    batch_inputs.append({
        "P": population,
        "I": intervention,
        "C": comparator,
        "O": outcome,
        "paper_content": paper,
        "criteria_text": criteria_text,
        "num_criteria": n_criteria
    })
            
LITERATURE_SCREENING_FC

'\n# CONTEXT #\nYou are a clinical specialist tasked with assessing research papers for inclusion in a meta-analysis based on specific eligibility criteria.\n\n# OBJECTIVE #\nEvaluate each criterion of a given paper to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly {num_criteria} responses.\n\n# IMPORTANT NOTE #\nIf the information within the provided paper content is insufficient to conclusively evaluate a criterion, you must opt for "UNCERTAIN" as your response. Avoid making assumptions or extrapolating beyond the provided data, as accurate and reliable responses are crucial, and fabricating information (hallucinations) could lead to serious errors in the meta-analysis.\n\n# PICO FRAMEWORK #\n- P (Patient, Problem or Population): {P}\n- I (Intervention): {I}\n- C (Comparison): {C}\n- O (Outcome): {O}\n\n# PAPER DETAILS #\n- Provided Paper: {paper_content}\

In [37]:
tools

[{'type': 'function',
  'function': {'name': 'PaperEvaluation',
   'description': '',
   'parameters': {'properties': {'evaluations': {'description': 'Evaluations for 4 criteria, must be of length 4',
      'items': {'enum': ['YES', 'NO', 'UNCERTAIN'], 'type': 'string'},
      'maxItems': 4,
      'minItems': 4,
      'type': 'array'},
     'rationale': {'description': 'A rationale for each criteria evaluation',
      'items': {'type': 'string'},
      'maxItems': 4,
      'minItems': 4,
      'type': 'array'}},
    'required': ['evaluations', 'rationale'],
    'type': 'object'}}}]

In [45]:
messages = _batch_inputs_to_messages(prompt_template=LITERATURE_SCREENING_FC, batch_inputs=batch_inputs)

from openai import AsyncOpenAI
tools = [{'type': 'function', 'function': {'name': 'PaperEvaluation', 'description': '', 'parameters': {'properties': {'evaluations': {'description': 'Evaluations for 4 criteria, must be of length 4', 'items': {'enum': ['YES', 'NO', 'UNCERTAIN'], 'type': 'string'}, 'maxItems': 4, 'minItems': 4, 'type': 'array'}, 'rationale': {'description': 'A rationale for each criteria evaluation', 'items': {'type': 'string'}, 'maxItems': 4, 'minItems': 4, 'type': 'array'}}, 'required': ['evaluations', 'rationale'], 'type': 'object'}}}]

async_openai_client = AsyncOpenAI(
        base_url=os.getenv("BASE_URL"),
        api_key=os.getenv("OPENAI_API_KEY"),
        http_client=httpx.AsyncClient(verify=False)
    )
response = await async_openai_client.chat.completions.create(
    model=os.getenv("MODEL_NAME"),
    messages=messages[0],
    tools=tools,
    temperature=0
)
response.choices[0].message

ChatCompletionMessage(content='```json\n{\n  "evaluations": [\n    "YES",\n    "YES",\n    "UNCERTAIN",\n    "YES",\n    "YES",\n    "UNCERTAIN"\n  ],\n  "rationale": [\n    "The title mentions \'metastatic colorectal cancer\'",\n    "The title mentions \'Crizotinib\'",\n    "There is no comparison mentioned in the title",\n    "Methods and results sections are briefly described but not detailed enough to confirm clarity",\n    "The text indicates adenocarcinoma as the type of colorectal cancer",\n    "No explicit statement about outcome duration is made"\n  ]\n}\n```  \n\n### Explanation of Decisions:\n1. **Does the title mention \'colorectal cancer\'?**  \n   *YES* — The title states “metastatic colorectal cancer.”\n   \n2. **Does the title mention \'crizotinib\'?**  \n   *YES* — The term “Crizotinib” appears in the title.\n\n3. **Is there any indication in the title that the study compares crizotinib to another treatment?**  \n   *UNCERTAIN* — There’s no direct comparison indicated 

In [52]:
response.choices[0].message.content

'```json\n{\n  "evaluations": [\n    "YES",\n    "YES",\n    "UNCERTAIN",\n    "YES",\n    "YES",\n    "UNCERTAIN"\n  ],\n  "rationale": [\n    "The title mentions \'metastatic colorectal cancer\'",\n    "The title mentions \'Crizotinib\'",\n    "There is no comparison mentioned in the title",\n    "Methods and results sections are briefly described but not detailed enough to confirm clarity",\n    "The text indicates adenocarcinoma as the type of colorectal cancer",\n    "No explicit statement about outcome duration is made"\n  ]\n}\n```  \n\n### Explanation of Decisions:\n1. **Does the title mention \'colorectal cancer\'?**  \n   *YES* — The title states “metastatic colorectal cancer.”\n   \n2. **Does the title mention \'crizotinib\'?**  \n   *YES* — The term “Crizotinib” appears in the title.\n\n3. **Is there any indication in the title that the study compares crizotinib to another treatment?**  \n   *UNCERTAIN* — There’s no direct comparison indicated in the title.\n\n4. **Are the 

In [60]:
from trialmind.api import extract_json,parse_json_outputs

In [62]:
parse_json_outputs([response.choices[0].message.content])

[{'evaluations': ['YES', 'YES', 'UNCERTAIN', 'YES', 'YES', 'UNCERTAIN'],
  'rationale': ["The title mentions 'metastatic colorectal cancer'",
   "The title mentions 'Crizotinib'",
   'There is no comparison mentioned in the title',
   'Methods and results sections are briefly described but not detailed enough to confirm clarity',
   'The text indicates adenocarcinoma as the type of colorectal cancer',
   'No explicit statement about outcome duration is made']}]

In [65]:

api = LiteratureScreening()
ec_pred = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    criteria = title_criteria + content_criteria,
    papers = papers[:1], # make for the top-100 for demo
)
ec_pred

{'name': 'PaperEvaluation', 'description': '', 'parameters': {'properties': {'evaluations': {'description': 'Evaluations for 6 criteria, must be of length 6', 'items': {'enum': ['YES', 'NO', 'UNCERTAIN'], 'type': 'string'}, 'maxItems': 6, 'minItems': 6, 'type': 'array'}, 'rationale': {'description': 'A rationale for each criteria evaluation', 'items': {'type': 'string'}, 'maxItems': 6, 'minItems': 6, 'type': 'array'}}, 'required': ['evaluations', 'rationale'], 'type': 'object'}}

api_function_call_single https://gigachat.devices.sberbank.ru/api/v1/
[{'role': 'user', 'content': '\n# CONTEXT #\nYou are a clinical specialist tasked with assessing research papers for inclusion in a meta-analysis based on specific eligibility criteria.\n\n# OBJECTIVE #\nEvaluate each criterion of a given paper to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly 6 responses.\n\n# IM

[['YES', 'YES', 'UNCERTAIN', 'YES', 'YES', 'UNCERTAIN']]

# d

In [9]:
pmid_list = df_papers[0].PMID.values.tolist()
#['41213063',#'26451310',]

res = pmid2biocxml(pmid_list, api_key=pubmed_api_key)
#print(res)
res = [parse_bioc_xml(r) for r in res]
# transform the parsed xml into paper content
papers = []
for parsed in res:
    paper_content = []
    for parsed_ in parsed["passage"]:
        paper_content.append(parsed_['content'])
    paper_content = "\n".join(paper_content)
    papers.append(paper_content)


https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41268440/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41213063/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40526090/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40211189/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40369167/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908


In [11]:
[len(i) for i in papers]

[1672, 1652, 27171, 2627, 46819]

In [15]:
splited = split_text_into_chunks(papers[1], chunk_size=600,
                                             chunk_overlap=300)
splited

['Case Report: HGF and NF1 mutations as putative bypass mechanisms of MET inhibitor resistance in hepatocellular carcinoma: a case study.',
 'Background: Hepatocellular carcinoma (HCC) is a highly aggressive liver cancer with poor prognosis, often associated with resistance to treatment. MET amplification has been identified as a potential therapeutic target, but resistance to MET inhibitors, such as crizotinib, remains a significant challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic',
 'challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression

In [12]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=papers,
    fields=['Results, string, main results of the study regarding Crizotinib, usually about treatment effectiveness'],#api.DEFAULT_FIELDS,
    llm='Qwen/Qwen3-32B',#"openai-gpt-4o",
    chunk_size=600,
    chunk_overlap=300,
    thinking=False,
)
extracted

False


[[{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [2],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic analysis using next-generation sequencing (NGS) was performed on biopsy samples taken before and after progression to identify mutations associated with resistance. Results: NGS revealed the loss of MET amplification and identified HGF and NF1 mutations as potential bypass mechanisms. Specifically,']}],
 [{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [5],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative rev

[[{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [4],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative reverse transcription polymerase chain reaction (qRT-PCR). There were 10 CRC-related RRAGs found. Prognostic models were built with RRAGs. Based on immune infiltration analysis, low-risk populations showed significantly greater levels of immune infiltration (P P ARHGEF17 mutation rate was 6%. Medications such as CGP-082996, Dasatinib, Erlotinib, and Salubrinal were more sensitive to high-risk group, whereas drugs such as FTI-277, DMOG, and Crizotinib were more sensitive to low-risk group.']}],
 [{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [1],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in ME

In [38]:
llm_reasoning = ChatOpenAI(
    base_url=BASE_URL,
    api_key=HUGGINGFACE_HUB_TOKEN,
    model=MODEL_NAME_HF,
    streaming=True,
    temperature=0,
    #http_client=httpx_client
    http_async_client=httpx.AsyncClient(verify=False)
)

In [47]:
prompt = ChatPromptTemplate.from_messages([
    ("user", '/no_think {text}'),
    # ("ai", "Good.")
])

chain = prompt | llm_reasoning
response = await chain.ainvoke({"text": ff})

#fin_condition = response.content.strip('<think>\n\n</think>\n\n')
print(response.content)

<think>

</think>

```json
[
    {
        "name": "Crizotinib effectiveness",
        "value": "More sensitive to low-risk group",
        "source_id": [3]
    }
]
```
